# Simulador de Energia Solar - Interface Interativa

Configure os parâmetros abaixo e execute a simulação!

In [14]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

# Importar funções do main.py
from main import simular_solar, exibir_resultados

sns.set_style('whitegrid')
%matplotlib inline

## Parâmetros da Simulação

In [15]:
# Parâmetros do Sistema Solar
tarifa_widget = widgets.FloatText(
    value=0.973,
    description='Tarifa (R$/kWh):',
    style={'description_width': 'initial'}
)

fio_b_widget = widgets.FloatText(
    value=0.69568,
    description='Fio B (R$/kWh):',
    style={'description_width': 'initial'}
)

geracao_widget = widgets.FloatText(
    value=1346.64,
    description='Geração mensal (kWh):',
    style={'description_width': 'initial'}
)

consumo_widget = widgets.FloatText(
    value=1237.17,
    description='Consumo mensal (kWh):',
    style={'description_width': 'initial'}
)

autoconsumo_widget = widgets.FloatSlider(
    value=0.20,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Autoconsumo (%):',
    style={'description_width': 'initial'},
    readout_format='.0%'
)

# Parâmetros Financeiros
parcela_widget = widgets.FloatText(
    value=750,
    description='Parcela (R$/mês):',
    style={'description_width': 'initial'}
)

meses_financ_widget = widgets.IntSlider(
    value=72,
    min=12,
    max=180,
    step=6,
    description='Financiamento (meses):',
    style={'description_width': 'initial'}
)

manutencao_widget = widgets.FloatText(
    value=750,
    description='Manutenção anual (R$):',
    style={'description_width': 'initial'}
)

# Parâmetros de Simulação
anos_widget = widgets.IntSlider(
    value=10,
    min=1,
    max=25,
    step=1,
    description='Anos de simulação:',
    style={'description_width': 'initial'}
)

reajuste_widget = widgets.FloatSlider(
    value=0.08,
    min=0.0,
    max=0.20,
    step=0.01,
    description='Reajuste anual (%):',
    style={'description_width': 'initial'},
    readout_format='.0%'
)

degradacao_widget = widgets.FloatSlider(
    value=0.005,
    min=0.0,
    max=0.02,
    step=0.001,
    description='Degradação anual (%):',
    style={'description_width': 'initial'},
    readout_format='.1%'
)

sazonalidade_widget = widgets.Checkbox(
    value=False,
    description='Usar sazonalidade (Florianópolis)',
    style={'description_width': 'initial'}
)

# Botão para executar
botao_simular = widgets.Button(
    description='🚀 Simular',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px')
)

# Organizar widgets em abas
tab_sistema = widgets.VBox([
    tarifa_widget,
    fio_b_widget,
    geracao_widget,
    consumo_widget,
    autoconsumo_widget
])

tab_financeiro = widgets.VBox([
    parcela_widget,
    meses_financ_widget,
    manutencao_widget
])

tab_simulacao = widgets.VBox([
    anos_widget,
    reajuste_widget,
    degradacao_widget,
    sazonalidade_widget
])

tabs = widgets.Tab(children=[tab_sistema, tab_financeiro, tab_simulacao])
tabs.set_title(0, '⚡ Sistema Solar')
tabs.set_title(1, '💰 Financeiro')
tabs.set_title(2, '📊 Simulação')

display(tabs)
display(botao_simular)

Button(button_style='success', description='🚀 Simular', layout=Layout(height='50px', width='200px'), style=But…

## Executar Simulação e Visualizar Resultados

In [16]:
# Área de saída
output_area = widgets.Output()

def executar_simulacao(b):
    with output_area:
        output_area.clear_output(wait=True)
        
        print("🔄 Executando simulação...\n")
        
        # Executar simulação
        df = simular_solar(
            tarifa_inicial=tarifa_widget.value,
            fio_b_inicial=fio_b_widget.value,
            geracao_mensal_media=geracao_widget.value,
            consumo_mensal=consumo_widget.value,
            perc_autoconsumo=autoconsumo_widget.value,
            valor_parcela=parcela_widget.value,
            meses_financiamento=meses_financ_widget.value,
            anos_simulacao=anos_widget.value,
            reajuste_anual=reajuste_widget.value,
            custo_manutencao_anual=manutencao_widget.value,
            degradacao_anual=degradacao_widget.value,
            usar_sazonalidade=sazonalidade_widget.value
        )
        
        # Estatísticas rápidas
        print("✅ Simulação concluída!\n")
        print("="*80)
        print("📊 RESUMO EXECUTIVO")
        print("="*80)
        print(f"💰 Fluxo líquido acumulado: R$ {df['Acumulado (R$)'].iloc[-1]:,.2f}")
        print(f"📈 Total economizado: R$ {df['Economia (R$)'].sum():,.2f}")
        print(f"💳 Total pago (financiamento): R$ {df['Parcela (R$)'].sum():,.2f}")
        print(f"🔧 Total manutenção: R$ {df['Manutenção (R$)'].sum():,.2f}")
        print(f"⚡ Total compra rede: R$ {df['Compra rede (R$)'].sum():,.2f}")
        print("="*80)
        
        # Gráfico do acumulado
        print("\n📈 Fluxo Líquido Acumulado\n")
        
        fig, ax = plt.subplots(figsize=(14, 6))
        
        # Plotar linha principal
        ax.plot(df['Mês'], df['Acumulado (R$)'], linewidth=3, color='#2ecc71', label='Acumulado')
        
        # Linha de referência zero
        ax.axhline(y=0, color='red', linestyle='--', alpha=0.7, linewidth=2)
        
        # Preencher áreas
        ax.fill_between(df['Mês'], df['Acumulado (R$)'], 0, 
                        where=(df['Acumulado (R$)'] >= 0), alpha=0.3, color='green', label='Positivo')
        ax.fill_between(df['Mês'], df['Acumulado (R$)'], 0, 
                        where=(df['Acumulado (R$)'] < 0), alpha=0.3, color='red', label='Negativo')
        
        # Marcar fim do financiamento
        if meses_financ_widget.value < len(df):
            ax.axvline(x=meses_financ_widget.value, color='purple', linestyle=':', 
                      linewidth=2, alpha=0.7, label=f'Fim financiamento (mês {meses_financ_widget.value})')
        
        ax.set_xlabel('Mês', fontsize=12)
        ax.set_ylabel('R$', fontsize=12)
        ax.set_title('Fluxo Líquido Acumulado ao Longo do Tempo', fontsize=14, fontweight='bold')
        ax.legend(loc='best', fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Formatar eixo Y com separador de milhares
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x:,.0f}'))
        
        plt.tight_layout()
        plt.show()
        
        # Mostrar tabela completa
        print("\n📋 TABELA COMPLETA DA SIMULAÇÃO\n")
        
        # Formatar DataFrame para exibição
        df_display = df.copy()
        
        # Formatar colunas monetárias
        for col in ['Economia (R$)', 'Compra rede (R$)', 'Parcela (R$)', 
                    'Manutenção (R$)', 'Fluxo líquido (R$)', 'Acumulado (R$)']:
            if col in df_display.columns:
                df_display[col] = df_display[col].apply(lambda x: f'R$ {x:,.2f}')
        
        # Formatar colunas de kWh
        for col in ['Geração (kWh)', 'Consumo (kWh)', 'Saldo créditos (kWh)']:
            if col in df_display.columns:
                df_display[col] = df_display[col].apply(lambda x: f'{x:,.2f}')
        
        # Formatar colunas de tarifa
        for col in ['Tarifa (R$/kWh)', 'Preço crédito (R$/kWh)']:
            if col in df_display.columns:
                df_display[col] = df_display[col].apply(lambda x: f'R$ {x:.3f}')
        
        display(HTML(df_display.to_html(index=False, escape=False)))

botao_simular.on_click(executar_simulacao)
display(output_area)

Output()